In [ ]:
import random

# Durasi tugas dan batasan
TUGAS = {
    'A': 2,  # A: 2 jam
    'B': 3,  # B: 3 jam
    'C': 2,  # C: 2 jam
    'D': 4   # D: 4 jam
}
MAKS_DURASI = 8

# Inisialisasi individu acak (urutan tugas)
def generate_individual():
    schedule = list(TUGAS.keys())
    random.shuffle(schedule)
    return schedule

# Inisialisasi populasi acak
def generate_population(size):
    return [generate_individual() for _ in range(size)]

# Hitung fitness dengan penalti bertahap
def fitness(individual):
    total_fitness = MAKS_DURASI  # Mulai dari nilai maksimal

    # Penalti jika A tidak sebelum B
    if individual.index('A') > individual.index('B'):
        total_fitness -= 2  # Penalti 2 jika A tidak sebelum B

    # Penalti jika C dan D terjadi bersamaan
    for i in range(len(individual) - 1):
        if (individual[i] == 'C' and individual[i + 1] == 'D') or (individual[i] == 'D' and individual[i + 1] == 'C'):
            total_fitness -= 2  # Penalti 2 jika C dan D bersebelahan

    # Hitung total durasi dan berikan penalti jika melebihi batas
    total_durasi = sum(TUGAS[task] for task in individual)
    if total_durasi > MAKS_DURASI:
        total_fitness -= (total_durasi - MAKS_DURASI)  # Penalti proporsional terhadap kelebihan durasi

    return max(0, total_fitness)  # Pastikan fitness tidak negatif

# Seleksi dengan Roulette Wheel Selection
def roulette_wheel_selection(population):
    fitness_scores = [fitness(ind) for ind in population]
    max_fitness = sum(fitness_scores)
    if max_fitness == 0:
        return random.choice(population)  # Jika semua fitness 0, pilih acak
    pick = random.uniform(0, max_fitness)
    current = 0

    for individual, score in zip(population, fitness_scores):
        current += score
        if current > pick:
            return individual

# Crossover: Gabungkan dua individu untuk membuat anak
def crossover(parent1, parent2):
    point = random.randint(1, len(TUGAS) - 2)
    child = parent1[:point] + [task for task in parent2 if task not in parent1[:point]]
    return child

# Mutasi: Tukar dua tugas dalam jadwal
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(individual)), 2)
        individual[i], individual[j] = individual[j], individual[i]
    return individual

# Algoritma Genetika dengan tampilan hasil akhir
def genetic_algorithm(pop_size=20, mutation_rate=0.1, max_generations=100):
    # Bangkitkan populasi awal
    population = generate_population(pop_size)

    for generation in range(max_generations):
        # Urutkan populasi berdasarkan fitness dan pertahankan individu terbaik (elitisme)
        population = sorted(population, key=fitness, reverse=True)
        best_individual = population[0]

        # Tampilkan informasi generasi
        print(f"Generasi {generation}: Jadwal terbaik: {best_individual}, Fitness: {fitness(best_individual)}")

        # Cek jika ditemukan individu dengan fitness maksimum (maksimal durasi yang diizinkan)
        if fitness(best_individual) >= MAKS_DURASI - 1:  # Cek solusi optimal atau mendekati optimal
            print(f"\nJadwal optimal ditemukan pada Generasi {generation}: {best_individual}")
            break

        # Membentuk populasi baru dengan mempertahankan individu terbaik
        new_population = population[:2]  # Pertahankan 2 individu terbaik dari generasi sebelumnya

        # Seleksi, crossover, dan mutasi untuk membentuk populasi baru
        while len(new_population) < pop_size:
            parent1 = roulette_wheel_selection(population)
            parent2 = roulette_wheel_selection(population)

            # Pastikan parent1 dan parent2 tidak sama
            while parent2 == parent1:
                parent2 = roulette_wheel_selection(population)

            # Crossover
            child1 = crossover(parent1, parent2)
            child2 = crossover(parent2, parent1)

            # Mutasi
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        # Update populasi dengan populasi baru
        population = new_population

    else:
        print("Tidak menemukan jadwal optimal dalam jumlah generasi maksimum.")

    # Tampilkan hasil akhir
    print("\nHasil Algoritma Genetika:")
    print(f"Solusi Terbaik: {population[0]}")
    print(f"Fitness Terbaik: {fitness(population[0])}")

# Jalankan algoritma
genetic_algorithm()


In [1]:
import random

# Definisikan durasi setiap kegiatan
durasi = {'A': 2, 'B': 3, 'C': 2, 'D': 4}
max_duration = 8  # Durasi maksimal

# Fungsi untuk menghitung fitness
def fitness(schedule):
    # Cek apakah A sebelum B
    if schedule.index('A') > schedule.index('B'):
        return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    # Cek apakah C dan D tidak bersebelahan
    for i in range(len(schedule) - 1):
        if (schedule[i] == 'C' and schedule[i + 1] == 'D') or (schedule[i] == 'D' and schedule[i + 1] == 'C'):
            return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    # Hitung total durasi dengan aturan penggabungan
    total_duration = 0
    gabung = [i for i in range(len(schedule)) if i % 2 == 0]

    # Menggabungkan durasi kegiatan dengan aturan penggabungan
    for i in gabung:
        if i + 1 < len(schedule):
            total_duration += max(durasi[schedule[i]], durasi[schedule[i+1]])
        else:
            total_duration += durasi[schedule[i]]  # Tambahkan jika tersisa satu tugas

    # Cek apakah total durasi melebihi batas
    if total_duration > max_duration:
        return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    return total_duration  # Semakin kecil total durasi, semakin baik

# Fungsi untuk membuat jadwal acak
def random_schedule():
    return random.sample(list(durasi.keys()), len(durasi))

# Algoritma Genetika
population_size = 20
generations = 50
population = [random_schedule() for _ in range(population_size)]

for generation in range(generations):
    print(f"Generasi {generation + 1}")

    # Hitung fitness setiap jadwal dan urutkan
    population = sorted(population, key=fitness, reverse=False)
    for pop in population:
        print(f"individu = {pop}, fitness = {fitness(pop)}", end="\n")

    # Seleksi jadwal terbaik
    population = population[:10]

    # Buat generasi baru dengan crossover dan mutasi
    new_population = []

    for _ in range(population_size):
        parent1, parent2 = random.sample(population, 2)
        cut = random.randint(1, len(durasi) - 1)
        child = parent1[:cut] + [x for x in parent2 if x not in parent1[:cut]]

        # Mutasi: Tukar dua kegiatan secara acak di dalam child
        if random.random() < 0.2:  # Meningkatkan tingkat mutasi
            i, j = random.sample(range(len(child)), 2)
            child[i], child[j] = child[j], child[i]

        new_population.append(child)

    # Tambahkan elitisme: Mempertahankan individu terbaik
    population = new_population + population[:2]  # Menambahkan 2 terbaik ke populasi baru
    population = sorted(population, key=fitness, reverse=False)[:population_size]

    for pop in population:
        print(f"individu baru = {pop}, fitness = {fitness(pop)}", end="\n")

# Jadwal yang memenuhi semua syarat:
population = sorted(population, key=fitness, reverse=False)

# Hanya jadwal unik yang memenuhi syarat
unique_schedules = []
for schedule in population:
    if schedule not in unique_schedules:
        unique_schedules.append(schedule)

# Print jadwal yang memenuhi semua syarat
for schedule in unique_schedules:
    if fitness(schedule) < 10:
        print("jadwal: ", schedule, end=" ")
        print("fitness: ", fitness(schedule))

# Cetak solusi terbaik
best_schedule = min(unique_schedules, key=fitness)
print("Jadwal terbaik:", best_schedule)
print("Fitness:", fitness(best_schedule))

Generasi 1
individu = ['C', 'A', 'B', 'D'], fitness = 6
individu = ['C', 'A', 'B', 'D'], fitness = 6
individu = ['C', 'A', 'B', 'D'], fitness = 6
individu = ['C', 'A', 'D', 'B'], fitness = 6
individu = ['A', 'D', 'B', 'C'], fitness = 7
individu = ['D', 'A', 'B', 'C'], fitness = 7
individu = ['D', 'B', 'C', 'A'], fitness = 10
individu = ['A', 'C', 'D', 'B'], fitness = 10
individu = ['D', 'C', 'A', 'B'], fitness = 10
individu = ['D', 'C', 'B', 'A'], fitness = 10
individu = ['A', 'D', 'C', 'B'], fitness = 10
individu = ['B', 'C', 'A', 'D'], fitness = 10
individu = ['A', 'D', 'C', 'B'], fitness = 10
individu = ['C', 'D', 'A', 'B'], fitness = 10
individu = ['B', 'A', 'C', 'D'], fitness = 10
individu = ['B', 'D', 'C', 'A'], fitness = 10
individu = ['C', 'B', 'A', 'D'], fitness = 10
individu = ['D', 'B', 'C', 'A'], fitness = 10
individu = ['D', 'C', 'A', 'B'], fitness = 10
individu = ['B', 'D', 'C', 'A'], fitness = 10
individu baru = ['C', 'A', 'D', 'B'], fitness = 6
individu baru = ['C', 'A'

In [4]:
import random

# Definisikan durasi setiap kegiatan
durasi = {'A': 2, 'B': 3, 'C': 2, 'D': 4}
max_duration = 8  # Durasi maksimal

# Fungsi untuk menghitung fitness
def fitness(schedule):
    # Cek apakah A sebelum B
    if schedule.index('A') > schedule.index('B'):
        return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    # Cek apakah C dan D tidak bersebelahan
    for i in range(len(schedule) - 1):
        if (schedule[i] == 'C' and schedule[i + 1] == 'D') or (schedule[i] == 'D' and schedule[i + 1] == 'C'):
            return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    # Hitung total durasi dengan aturan penggabungan
    total_duration = 0
    gabung = [i for i in range(len(schedule)) if i % 2 == 0]

    # Menggabungkan durasi kegiatan dengan aturan penggabungan
    for i in gabung:
        if i + 1 < len(schedule):
            total_duration += max(durasi[schedule[i]], durasi[schedule[i+1]])
        else:
            total_duration += durasi[schedule[i]]  # Tambahkan jika tersisa satu tugas

    # Cek apakah total durasi melebihi batas
    if total_duration > max_duration:
        return 10  # penalty untuk jadwal yang tidak memenuhi aturan

    return total_duration  # Semakin kecil total durasi, semakin baik

# Fungsi untuk membuat jadwal acak
def random_schedule():
    return random.sample(list(durasi.keys()), len(durasi))

# Fungsi untuk menggabungkan jadwal menjadi pasangan seperti ['CA', 'BD']
def combine_schedule(schedule):
    combined_schedule = []
    for i in range(0, len(schedule), 2):
        pair = ''.join(schedule[i:i+2])  # Gabungkan dua elemen sekaligus
        combined_schedule.append(pair)
    return combined_schedule

# Algoritma Genetika
population_size = 20
generations = 50
population = [random_schedule() for _ in range(population_size)]

for generation in range(generations):
    print(f"Generasi {generation + 1}")

    # Hitung fitness setiap jadwal dan urutkan
    population = sorted(population, key=fitness, reverse=False)
    for pop in population:
        print(f"individu = {combine_schedule(pop)}, fitness = {fitness(pop)}", end="\n")

    # Seleksi jadwal terbaik
    population = population[:10]

    # Buat generasi baru dengan crossover dan mutasi
    new_population = []

    for _ in range(population_size):
        parent1, parent2 = random.sample(population, 2)
        cut = random.randint(1, len(durasi) - 1)
        child = parent1[:cut] + [x for x in parent2 if x not in parent1[:cut]]

        # Mutasi: Tukar dua kegiatan secara acak di dalam child
        if random.random() < 0.2:  # Meningkatkan tingkat mutasi
            i, j = random.sample(range(len(child)), 2)
            child[i], child[j] = child[j], child[i]

        new_population.append(child)

    # Tambahkan elitisme: Mempertahankan individu terbaik
    population = new_population + population[:2]  # Menambahkan 2 terbaik ke populasi baru
    population = sorted(population, key=fitness, reverse=False)[:population_size]

    for pop in population:
        print(f"individu baru = {combine_schedule(pop)}, fitness = {fitness(pop)}", end="\n")

# Jadwal yang memenuhi semua syarat:
population = sorted(population, key=fitness, reverse=False)

# Hanya jadwal unik yang memenuhi syarat
unique_schedules = []
for schedule in population:
    if schedule not in unique_schedules:
        unique_schedules.append(schedule)

# Print jadwal yang memenuhi semua syarat
for schedule in unique_schedules:
    if fitness(schedule) < 10:
        print("jadwal: ", combine_schedule(schedule), end=" ")
        print("fitness: ", fitness(schedule))

# Cetak solusi terbaik
best_schedule = min(unique_schedules, key=fitness)
print("Jadwal terbaik:", combine_schedule(best_schedule))
print("Fitness:", fitness(best_schedule))


Generasi 1
individu = ['AC', 'BD'], fitness = 6
individu = ['CA', 'BD'], fitness = 6
individu = ['DA', 'CB'], fitness = 7
individu = ['DA', 'BC'], fitness = 7
individu = ['DA', 'BC'], fitness = 7
individu = ['AD', 'BC'], fitness = 7
individu = ['DA', 'BC'], fitness = 7
individu = ['DA', 'CB'], fitness = 7
individu = ['DC', 'BA'], fitness = 10
individu = ['BD', 'AC'], fitness = 10
individu = ['DC', 'BA'], fitness = 10
individu = ['BD', 'AC'], fitness = 10
individu = ['CD', 'BA'], fitness = 10
individu = ['DB', 'AC'], fitness = 10
individu = ['CD', 'BA'], fitness = 10
individu = ['BA', 'CD'], fitness = 10
individu = ['BA', 'CD'], fitness = 10
individu = ['DC', 'BA'], fitness = 10
individu = ['BA', 'DC'], fitness = 10
individu = ['AB', 'CD'], fitness = 10
individu baru = ['CA', 'DB'], fitness = 6
individu baru = ['AC', 'BD'], fitness = 6
individu baru = ['CA', 'BD'], fitness = 6
individu baru = ['DA', 'CB'], fitness = 7
individu baru = ['DA', 'CB'], fitness = 7
individu baru = ['DA', 'BC'